In [402]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.width", 1000)
pd.options.display.float_format = "{:,.2f}".format

import random
import numpy as np
import json

PARAMETROS_SERIE = {
    'A': {'escala_gols': 13.0, 'fator_casa': 1.20},
    'B': {'escala_gols': 22.0, 'fator_casa': 1.25},
    'C': {'escala_gols': 28.0, 'fator_casa': 1.30}
}

In [403]:
random.seed(27)
np.random.seed(27)

df_clubes = pd.read_parquet(r"data/clubes.parquet")
df_calendario = pd.read_parquet(r"data/calendario.parquet")

clubes_serie_a = list(df_clubes['CLUBE'].iloc[0:20])
clubes_serie_b = list(df_clubes['CLUBE'].iloc[20:40])
clubes_serie_c = list(df_clubes['CLUBE'].iloc[40:60])

random.shuffle(clubes_serie_a)
random.shuffle(clubes_serie_b)
random.shuffle(clubes_serie_c)

mapa_clubes = {}

for i in range(20):
    placeholder = f"clube {i+1:02d}"
    clube_real = clubes_serie_a[i]
    mapa_clubes[placeholder] = clube_real

for i in range(20):
    placeholder = f"clube {i+21:02d}"
    clube_real = clubes_serie_b[i]
    mapa_clubes[placeholder] = clube_real
    
for i in range(20):
    placeholder = f"clube {i+41:02d}"
    clube_real = clubes_serie_c[i]
    mapa_clubes[placeholder] = clube_real

df_temporada_1 = df_calendario.copy()
df_temporada_1['mandante_real'] = df_temporada_1['mandante'].map(mapa_clubes)
df_temporada_1['visitante_real'] = df_temporada_1['visitante'].map(mapa_clubes)

print("--- Calendário Oficial da Temporada 1 (Exemplo) ---")
print(df_temporada_1[['serie', 'rodada', 'partida', 'mandante', 'mandante_real', 'visitante', 'visitante_real']].head(30))

--- Calendário Oficial da Temporada 1 (Exemplo) ---
   serie  rodada  partida  mandante         mandante_real visitante        visitante_real
0      A       1        1  clube 16           Corinthians  clube 18                Grêmio
1      A       1        2  clube 11           Santo André  clube 09   Red Bull Bragantino
2      A       1        3  clube 08               Guarani  clube 10  Athletico Paranaense
3      A       1        4  clube 01              Coritiba  clube 07           Ponte Preta
4      A       1        5  clube 04              Flamengo  clube 03      Atlético Mineiro
5      A       1        6  clube 13           São Caetano  clube 02              Botafogo
6      A       1        7  clube 15             Palmeiras  clube 05             São Paulo
7      A       1        8  clube 20                 Sport  clube 19                Santos
8      A       1        9  clube 17            Fluminense  clube 06         Vasco da Gama
9      A       1       10  clube 12         Inte

In [404]:
df_clubes_renomeado = df_clubes.rename(columns={'FORCA': 'forca_base'})
estado_clubes = df_clubes_renomeado.set_index('CLUBE').to_dict('index')

for clube in estado_clubes:
    estado_clubes[clube]['pontos'] = 0
    estado_clubes[clube]['vitorias'] = 0
    estado_clubes[clube]['empates'] = 0
    estado_clubes[clube]['derrotas'] = 0
    estado_clubes[clube]['gols_pro'] = 0
    estado_clubes[clube]['gols_contra'] = 0
    estado_clubes[clube]['momento'] = 1.0

# print(json.dumps(estado_clubes['São Paulo'], indent=4))

In [405]:
def simular_jogo(mandante_nome, visitante_nome, serie_jogo, estado_clubes, fator_equilibrio_classico=0.60, forca_minima=0.02):
    """
    Simula uma única partida entre dois clubes.
    Retorna um dicionário com o resultado e os placares.
    """
    
    parametros = PARAMETROS_SERIE[serie_jogo]
    escala_gols = parametros['escala_gols']
    fator_casa = parametros['fator_casa']

    mandante_dados = estado_clubes[mandante_nome]
    visitante_dados = estado_clubes[visitante_nome]

    forca_base_mandante_ajustada = mandante_dados['forca_base'] + forca_minima
    forca_base_visitante_ajustada = visitante_dados['forca_base'] + forca_minima
    
    forca_ataque_mandante = (forca_base_mandante_ajustada * escala_gols) * mandante_dados['momento'] * fator_casa
    forca_ataque_visitante = (forca_base_visitante_ajustada * escala_gols) * mandante_dados['momento']

    if mandante_dados['UF'] == visitante_dados['UF']:

        forca_media = (forca_ataque_mandante + forca_ataque_visitante) / 2
        forca_ataque_mandante = (forca_ataque_mandante * fator_equilibrio_classico) + (forca_media * (1 - fator_equilibrio_classico))
        forca_ataque_visitante = (forca_ataque_visitante * fator_equilibrio_classico) + (forca_media * (1 - fator_equilibrio_classico))

    gols_mandante = np.random.poisson(forca_ataque_mandante)
    gols_visitante = np.random.poisson(forca_ataque_visitante)
    
    if gols_mandante > gols_visitante:
        resultado = 'V'
    elif gols_visitante > gols_mandante:
        resultado = 'D'
    else:
        resultado = 'E'
        
    return {
        'resultado': resultado,
        'gols_mandante': gols_mandante,
        'gols_visitante': gols_visitante
    }

In [406]:
lista_de_resultados = []

for index, jogo in df_temporada_1.iterrows():
    mandante_nome = jogo['mandante_real']
    visitante_nome = jogo['visitante_real']
    serie_do_jogo = jogo['serie']
    
    resultado_partida = simular_jogo(mandante_nome, visitante_nome, serie_do_jogo, estado_clubes)

    fator_vitoria_fora = 1.07
    fator_vitoria_casa = 1.02

    fator_empate_fora = 1.03
    fator_empate_casa = 0.97

    fator_derrota_fora = 0.99
    fator_derrota_casa = 0.90

    min_momento = 0.85
    max_momento = 1.25
    
    lista_de_resultados.append(resultado_partida)
    
    estado_clubes[mandante_nome]['gols_pro'] += resultado_partida['gols_mandante']
    estado_clubes[mandante_nome]['gols_contra'] += resultado_partida['gols_visitante']
    estado_clubes[visitante_nome]['gols_pro'] += resultado_partida['gols_visitante']
    estado_clubes[visitante_nome]['gols_contra'] += resultado_partida['gols_mandante']
    
    if resultado_partida['resultado'] == 'V':
        estado_clubes[mandante_nome]['pontos'] += 3
        estado_clubes[mandante_nome]['vitorias'] += 1
        estado_clubes[mandante_nome]['momento'] *= fator_vitoria_casa
        
        estado_clubes[visitante_nome]['derrotas'] += 1
        estado_clubes[visitante_nome]['momento'] *= fator_derrota_fora

    elif resultado_partida['resultado'] == 'D':
        estado_clubes[visitante_nome]['pontos'] += 3
        estado_clubes[visitante_nome]['vitorias'] += 1
        estado_clubes[visitante_nome]['momento'] *= fator_vitoria_fora

        estado_clubes[mandante_nome]['derrotas'] += 1
        estado_clubes[mandante_nome]['momento'] *= fator_derrota_casa

    else:
        estado_clubes[mandante_nome]['pontos'] += 1
        estado_clubes[mandante_nome]['empates'] += 1
        estado_clubes[mandante_nome]['momento'] *= fator_empate_casa

        estado_clubes[visitante_nome]['pontos'] += 1
        estado_clubes[visitante_nome]['empates'] += 1
        
        estado_clubes[visitante_nome]['momento'] *= fator_empate_fora
    
    estado_clubes[mandante_nome]['momento'] = max(min_momento, min(estado_clubes[mandante_nome]['momento'], max_momento))
    estado_clubes[visitante_nome]['momento'] = max(min_momento, min(estado_clubes[visitante_nome]['momento'], max_momento))


df_resultados_t1 = pd.DataFrame(lista_de_resultados)
df_temporada_1_final = pd.concat([df_temporada_1, df_resultados_t1], axis=1)

In [407]:
teste = df_temporada_1_final[
  (df_temporada_1_final["mandante_real"] == "São Paulo")
  | (df_temporada_1_final["visitante_real"] == "São Paulo")
]

print("\n--- Temporada 1 Simulada com Resultados ---")
print(teste[['serie', 'rodada', 'mandante_real', 'visitante_real', 'gols_mandante', 'gols_visitante']])


--- Temporada 1 Simulada com Resultados ---
    serie  rodada         mandante_real        visitante_real  gols_mandante  gols_visitante
6       A       1             Palmeiras             São Paulo              1               2
15      A       2             São Paulo              Flamengo              3               1
24      A       3               Guarani             São Paulo              0               1
33      A       4             São Paulo                Grêmio              4               0
42      A       5  Athletico Paranaense             São Paulo              2               0
51      A       6             São Paulo      Atlético Mineiro              1               1
60      A       7           Corinthians             São Paulo              0               2
71      A       8         Vasco da Gama             São Paulo              0               1
82      A       9             São Paulo         Internacional              4               1
93      A      10        

In [408]:
df_classificacao = pd.DataFrame.from_dict(estado_clubes, orient='index')
df_classificacao['saldo_gols'] = df_classificacao['gols_pro'] - df_classificacao['gols_contra']
print("\n--- Classificação Final da Temporada 1 (Série A) ---")
print(df_classificacao[df_classificacao.index.isin(clubes_serie_a)].sort_values(by=['pontos', 'vitorias', 'saldo_gols'], ascending=False))


--- Classificação Final da Temporada 1 (Série A) ---
                      UF  forca_base  pontos  vitorias  empates  derrotas  gols_pro  gols_contra  momento  saldo_gols
São Paulo             SP        0.10      78        22       12         4        64           31     1.25          33
Flamengo              RJ        0.11      76        22       10         6        90           42     1.21          48
Palmeiras             SP        0.11      74        22        8         8        77           38     1.20          39
Corinthians           SP        0.10      73        22        7         9        63           48     1.25          15
Cruzeiro              MG        0.07      73        20       13         5        61           34     1.25          27
Internacional         RS        0.07      64        17       13         8        49           32     1.22          17
Atlético Mineiro      MG        0.06      63        17       12         9        45           36     1.25           9
Fl

In [409]:
print("\n--- Classificação Final da Temporada 1 (Série B) ---")
print(df_classificacao[df_classificacao.index.isin(clubes_serie_b)].sort_values(by=['pontos', 'vitorias', 'saldo_gols'], ascending=False))


--- Classificação Final da Temporada 1 (Série B) ---
                     UF  forca_base  pontos  vitorias  empates  derrotas  gols_pro  gols_contra  momento  saldo_gols
Juventude            RS        0.01      71        21        8         9        48           28     1.24          20
Vitória              BA        0.01      66        20        6        12        39           24     0.93          15
Fortaleza            CE        0.01      58        15       13        10        40           29     1.16          11
Botafogo-SP          SP        0.01      58        15       13        10        39           33     1.25           6
Bahia                BA        0.01      57        14       15         9        44           35     1.09           9
Paraná               PR        0.01      56        15       11        12        34           30     1.05           4
Ipatinga             MG        0.01      53        13       14        11        27           26     1.14           1
Grêmio Pru

In [410]:
print("\n--- Classificação Final da Temporada 1 (Série C) ---")
print(df_classificacao[df_classificacao.index.isin(clubes_serie_c)].sort_values(by=['pontos', 'vitorias', 'saldo_gols'], ascending=False))


--- Classificação Final da Temporada 1 (Série C) ---
                      UF  forca_base  pontos  vitorias  empates  derrotas  gols_pro  gols_contra  momento  saldo_gols
Paysandu              PA        0.00      68        19       11         8        42           21     1.22          21
Operário Ferroviário  PR        0.00      65        18       11         9        40           26     1.25          14
Atlético Acreano      AC        0.00      61        17       10        11        39           28     0.94          11
Santa Cruz            PE        0.00      59        15       14         9        34           25     1.15           9
Ypiranga-RS           RS        0.01      56        13       17         8        23           21     1.08           2
Remo                  PA        0.00      55        13       16         9        22           24     1.24          -2
ABC                   RN        0.00      54        13       15        10        29           22     1.05           7
Jo

In [411]:
# Adicione este bloco ao FINAL do seu script que simula uma temporada

print("\n--- PÓS-TEMPORADA: APLICANDO PROMOÇÃO E REBAIXAMENTO ---")

# Para garantir que temos a série de cada clube na tabela de classificação, vamos adicionar essa informação
df_classificacao['serie'] = pd.Series({
    clube: 'A' for clube in clubes_serie_a
} | {
    clube: 'B' for clube in clubes_serie_b
} | {
    clube: 'C' for clube in clubes_serie_c
})

# 1. Ordena a tabela de classificação completa para fácil acesso
df_classificacao_ordenada = df_classificacao.sort_values(
    by=['serie', 'pontos', 'saldo_gols'], 
    ascending=[True, False, False] # Ordem: Serie A, B, C; Mais pontos primeiro
)

# 2. Identifica os times que mudam de divisão
rebaixados_a = df_classificacao_ordenada.query("serie == 'A'").tail(4).index.tolist()
promovidos_b = df_classificacao_ordenada.query("serie == 'B'").head(4).index.tolist()

rebaixados_b = df_classificacao_ordenada.query("serie == 'B'").tail(4).index.tolist()
promovidos_c = df_classificacao_ordenada.query("serie == 'C'").head(4).index.tolist()

print(f"\nRebaixados da Série A: {rebaixados_a}")
print(f"Promovidos para a Série A: {promovidos_b}")
print(f"\nRebaixados da Série B: {rebaixados_b}")
print(f"Promovidos para a Série B: {promovidos_c}")


# 3. Atualiza as listas de clubes para a PRÓXIMA temporada
clubes_serie_a_prox_ano = [clube for clube in clubes_serie_a if clube not in rebaixados_a] + promovidos_b
clubes_serie_b_prox_ano = [clube for clube in clubes_serie_b if clube not in promovidos_b and clube not in rebaixados_b] + rebaixados_a + promovidos_c
clubes_serie_c_prox_ano = [clube for clube in clubes_serie_c if clube not in promovidos_c] + rebaixados_b

print("\n--- COMPOSIÇÃO DAS LIGAS PARA A PRÓXIMA TEMPORADA ---")
print(f"Nova Série A ({len(clubes_serie_a_prox_ano)} clubes): {clubes_serie_a_prox_ano}")


--- PÓS-TEMPORADA: APLICANDO PROMOÇÃO E REBAIXAMENTO ---

Rebaixados da Série A: ['São Caetano', 'Vasco da Gama', 'Red Bull Bragantino', 'Coritiba']
Promovidos para a Série A: ['Juventude', 'Vitória', 'Fortaleza', 'Botafogo-SP']

Rebaixados da Série B: ['Grêmio Barueri', 'Criciúma', 'Figueirense', 'Volta Redonda']
Promovidos para a Série B: ['Paysandu', 'Operário Ferroviário', 'Atlético Acreano', 'Santa Cruz']

--- COMPOSIÇÃO DAS LIGAS PARA A PRÓXIMA TEMPORADA ---
Nova Série A (20 clubes): ['Botafogo', 'Atlético Mineiro', 'Flamengo', 'São Paulo', 'Ponte Preta', 'Guarani', 'Athletico Paranaense', 'Santo André', 'Internacional', 'Cruzeiro', 'Palmeiras', 'Corinthians', 'Fluminense', 'Grêmio', 'Santos', 'Sport', 'Juventude', 'Vitória', 'Fortaleza', 'Botafogo-SP']


In [412]:
# Adicione este bloco após o de promoção/rebaixamento

print("\n--- ENTRESSAFRA: EVOLUINDO A FORÇA DOS CLUBES ---")

# Fatores de evolução (sinta-se à vontade para calibrar)
fator_campeao_a = 1.03
fator_promovido_para_a = 1.02
fator_rebaixado_da_a = 0.97

fator_campeao_b = 1.02
fator_promovido_para_b = 1.01
fator_rebaixado_da_b = 0.98

fator_campeao_c = 1.01

# Identifica os campeões
campeao_a = df_classificacao_ordenada.query("serie == 'A'").index[0]
campeao_b = df_classificacao_ordenada.query("serie == 'B'").index[0]
campeao_c = df_classificacao_ordenada.query("serie == 'C'").index[0]

# Aplica bônus e penalidades (trabalharemos no dicionário 'estado_clubes' que contém as forças)
estado_clubes[campeao_a]['forca_base'] *= fator_campeao_a
estado_clubes[campeao_b]['forca_base'] *= fator_campeao_b
estado_clubes[campeao_c]['forca_base'] *= fator_campeao_c

for clube in promovidos_b: # Promovidos para a Série A (que não foram campeões da B)
    if clube != campeao_b:
        estado_clubes[clube]['forca_base'] *= fator_promovido_para_a
for clube in rebaixados_a:
    estado_clubes[clube]['forca_base'] *= fator_rebaixado_da_a

for clube in promovidos_c: # Promovidos para a Série B (que não foram campeões da C)
    if clube != campeao_c:
        estado_clubes[clube]['forca_base'] *= fator_promovido_para_b
for clube in rebaixados_b:
    estado_clubes[clube]['forca_base'] *= fator_rebaixado_da_b

# Aplicando a leve regressão à média para todos
forca_media_geral = df_classificacao['forca_base'].mean()
for clube in estado_clubes:
    # Puxa a força de cada time 1% em direção à média geral
    estado_clubes[clube]['forca_base'] = estado_clubes[clube]['forca_base'] * 0.99 + forca_media_geral * 0.01

print(f"\nForça do {campeao_a} (campeão da A) para o próximo ano: {estado_clubes[campeao_a]['forca_base']:.4f}")
print(f"Força do {rebaixados_a[0]} (rebaixado da A) para o próximo ano: {estado_clubes[rebaixados_a[0]]['forca_base']:.4f}")
print(f"Força do {rebaixados_a[1]} (rebaixado da A) para o próximo ano: {estado_clubes[rebaixados_a[1]]['forca_base']:.4f}")


--- ENTRESSAFRA: EVOLUINDO A FORÇA DOS CLUBES ---

Força do São Paulo (campeão da A) para o próximo ano: 0.0971
Força do São Caetano (rebaixado da A) para o próximo ano: 0.0150
Força do Vasco da Gama (rebaixado da A) para o próximo ano: 0.0295


In [ ]:
teste = pd.read_parquet(r"data/simulacao_40_anos.parquet")